# <center> Environment Project </center> 
### <center> Analysis of environment data </center>

In this work we will analyze various types of data related to pollution. We will try to obtain some conclusions, try to answer some question, in order to achieve this, the data will need treatment like nulls filling ,  outliers removal, in order to delevop this results, visualizing and observing the data will be key. 

For this project we are using the libraries _pandas_ for data management,  _seaborn_ for data viuslization and _geopy_ to calculate distances between different locations

In [196]:
import pandas as pd
import seaborn as sns
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

The data that we used was :  
- _Air Quality_ : It's a data-set from the city of London with the principal componentes of the air, which defines the quality of it.The data consists in  a mean of all days and it's group by month. Can be found at:  ?  

- _Pollution_ : This data-set contains the quantity of pollutant gases emited to the air per year. Many countries are included there.  
Can be found at:  https://www.eea.europa.eu/en/datahub/datahubitem-view/9405f714-8015-4b5b-a63c-280b82861b3d  

- _Traffic flow_ : Here we can find the traffic flow from different regions of London per year, distinguished for the  different roads of it.  
Can be found at: https://roadtraffic.dft.gov.uk/regions/6  

- _Health index_ : The last set of data represents the health index. Instead of of having a single index, it contains  one for each area . For exemaple, if one region has a lot of people that do exercise the health index will be postive in that area.  
Can be found at: https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/datasets/healthindexunderlyingdataengland

### The first step is to load and filtrate the data for a area of 50 km of radius around the city of London

In [197]:
#Loading of the data

#air quality file
data_air = pd.read_csv("datasets/air-quality-london-monthly-averages.csv")

#pollution file
data_pol = pd.read_csv("datasets/pollution.csv")

#traffic flow file
data_traf = pd.read_csv("datasets/dft_aadf_region_id_6.csv")

#health index file
data_health = pd.read_csv("datasets/healthindexunderlyingdataenglandcanscreenfix.csv",delimiter=";")

C:\Users\joaop\AppData\Local\Temp\ipykernel_19740\830281676.py:7: DtypeWarning: Columns (7,10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_pol = pd.read_csv("datasets/pollution.csv")


#### Before the data filtering, we must visualize it on a table format to see all the variables and how their values are disposed 

The air and traffic flow data-set just includes the London city, thus there's no need to aplly this first filtering

In [198]:
data_air.head()

,Month,London Mean Roadside:Nitric Oxide (ug/m3),London Mean Roadside:Nitrogen Dioxide (ug/m3),London Mean Roadside:Oxides of Nitrogen (ug/m3),London Mean Roadside:Ozone (ug/m3),London Mean Roadside:PM10 Particulate (ug/m3),London Mean Roadside:PM2.5 Particulate (ug/m3),London Mean Roadside:Sulphur Dioxide (ug/m3),London Mean Background:Nitric Oxide (ug/m3),London Mean Background:Nitrogen Dioxide (ug/m3),London Mean Background:Oxides of Nitrogen (ug/m3),London Mean Background:Ozone (ug/m3),London Mean Background:PM10 Particulate (ug/m3),London Mean Background:PM2.5 Particulate (ug/m3),London Mean Background:Sulphur Dioxide (ug/m3)
0,Jan-08,NaN,55.5,NaN,29.5,25.0,14.7,4.2,NaN,42.3,NaN,36.9,18.8,.,3.6
1,Feb-08,NaN,75.9,NaN,20.3,39.5,28.8,7.6,NaN,60.2,NaN,26.4,31.9,.,6.7
2,Mar-08,NaN,55.6,NaN,40.1,21.6,12.3,3.9,NaN,39.8,NaN,50.2,15.5,.,2.3
3,Apr-08,NaN,61.8,NaN,37.9,28.7,20.5,4.5,NaN,44.0,NaN,50.1,21.7,.,3.2
4,May-08,NaN,62.9,NaN,46.3,34.6,27.5,4.6,NaN,44.1,NaN,60.5,29.5,16.6,4.3


In [199]:
data_traf.head()

,count_point_id,year,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,6750,2014,6,London,96,Islington,A201,Major,Swinton St,Pentonville Rd,...,787,1728,204,74,132,15,6,18,449,11916
1,8352,2014,6,London,104,Lewisham,A2,Major,A20,A20,...,430,1415,258,36,58,6,11,22,390,9426
2,6807,2014,6,London,178,Kingston upon Thames,A240,Major,A3210,A307,...,175,1589,251,35,13,2,4,19,325,12951
3,27765,2014,6,London,57,Barnet,A5109,Major,Barnet Lane,A1000/A109,...,302,1651,246,12,12,0,1,1,273,19276
4,8488,2014,6,London,104,Lewisham,A20,Major,A20 Amersham Rd,A2 Parkfield Rd,...,656,1617,286,83,27,5,8,7,415,12821


In [200]:
data_pol.head()

,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
0,Austria,4.0,Chemical industry,4(d),Chemical installations for the production on a...,AT.CAED/9008390392737.FACILITY,Nufarm GmbH & CO KG,NaN,14.336056,48.290943,NaN,Linz,AIR,"1,2-dichloroethane (DCE-1,2)",3300.0,2007,NaN
1,Austria,9.0,Other activities,9(c),Installations for the surface treatment of sub...,AT.CAED/9008390396414.FACILITY,MAGNA STEYR Fahrzeugtechnik AG&Co KG,NaN,15.477778,47.019170,NaN,"Graz,07.Bez.:Liebenau",AIR,Non-methane volatile organic compounds (NMVOC),170000.0,2007,NaN
2,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Carbon dioxide (CO2),228000000.0,2007,NaN
3,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Nitrogen oxides (NOX),311000.0,2007,NaN
4,Austria,3.0,Mineral industry,3(c),Installations for the production of cement cli...,AT.CAED/9008390397220.FACILITY,voestalpine Stahl GmbH,NaN,14.159778,47.807083,NaN,Klaus an der Pyhrnbahn,AIR,Carbon dioxide (CO2),329000000.0,2007,NaN


In [201]:
def citiesWithinRadius(cities,radius,data_cities,data_from_outside=True):
    lat_lon_london = (51.509865, -0.118092)
    cidades = []
    #iterate all cities around london
    for city in cities:
        cidade = city
        #check if the data countains coordinates information or not and take the row with city iterated
        if data_from_outside:
            city = data_cities[data_cities["city"]== city]
        else:
            city = data_cities[data_cities["City"]== city]

        try:
            #take the corrdinates
            if data_from_outside:
                loc = (pd.array(city["lat"])[0],pd.array(city["lng"])[0])
            else:
                loc = (pd.array(city["Latitude"])[0],pd.array(city["Longitude"])[0])
        except:
            continue
        #if the distance is lower than 50, append to the list
        distance = geodesic(lat_lon_london, loc).kilometers
        if distance<=50:
            cidades.append(cidade)
    return cidades

In [202]:
#just Uk country
data_cities = data_pol[data_pol["countryName"]=="United Kingdom"]
#return all cities within a radius of 50 km from London
city_50km = citiesWithinRadius(data_health["Area name"].unique(), 50, data_cities,False)
#select the cities around london
data_pol = data_pol[data_pol["City"].isin(city_50km)]
data_health.head()


,Area code,Area name,Value,Numerator,Denominator,Indicator name,Year
0,E06000001,Hartlepool,"62,68947549",NaN,NaN,Physical activity,2015
1,E06000002,Middlesbrough,"62,8371136",NaN,NaN,Physical activity,2015
2,E06000003,Redcar and Cleveland,"65,80605787",NaN,NaN,Physical activity,2015
3,E06000004,Stockton-on-Tees,"59,83048525",NaN,NaN,Physical activity,2015
4,E06000005,Darlington,"65,39884288",NaN,NaN,Physical activity,2015


In [203]:
data_health.head()

,Area code,Area name,Value,Numerator,Denominator,Indicator name,Year
0,E06000001,Hartlepool,"62,68947549",NaN,NaN,Physical activity,2015
1,E06000002,Middlesbrough,"62,8371136",NaN,NaN,Physical activity,2015
2,E06000003,Redcar and Cleveland,"65,80605787",NaN,NaN,Physical activity,2015
3,E06000004,Stockton-on-Tees,"59,83048525",NaN,NaN,Physical activity,2015
4,E06000005,Darlington,"65,39884288",NaN,NaN,Physical activity,2015


The data-set of the cities with their locations was found in : https://simplemaps.com/data/world-cities

In [204]:
#information of the cities coordinates
data_cities = pd.read_csv("datasets/worldcities.csv")

data_cities = data_cities[data_cities["country"]=="United Kingdom"]
city_50km = citiesWithinRadius(data_health["Area name"].unique(), 50, data_cities)
data_health = data_health[data_health["Area name"].isin(city_50km)]
data_health.head()

,Area code,Area name,Value,Numerator,Denominator,Indicator name,Year
29,E06000032,Luton,"61,75566862",NaN,NaN,Physical activity,2015
36,E06000039,Slough,"59,60549999",NaN,NaN,Physical activity,2015
90,E07000066,Basildon,"65,4596393",NaN,NaN,Physical activity,2015
92,E07000068,Brentwood,"68,62717127",NaN,NaN,Physical activity,2015
94,E07000070,Chelmsford,"67,26048083",NaN,NaN,Physical activity,2015


### Not every variable is relevant to the analysis,thus some of them will be excluded  
Also, there will be a modification in the name of the columns, to simpler one and a re-indexing, where the dates will be the "primary key", so, in the future, we are allowed to merge with other data-sets

Air quality is determined by the presence and concentration of various air pollutant, the most common ones will be included.    


In [205]:
data_air.head()

,Month,London Mean Roadside:Nitric Oxide (ug/m3),London Mean Roadside:Nitrogen Dioxide (ug/m3),London Mean Roadside:Oxides of Nitrogen (ug/m3),London Mean Roadside:Ozone (ug/m3),London Mean Roadside:PM10 Particulate (ug/m3),London Mean Roadside:PM2.5 Particulate (ug/m3),London Mean Roadside:Sulphur Dioxide (ug/m3),London Mean Background:Nitric Oxide (ug/m3),London Mean Background:Nitrogen Dioxide (ug/m3),London Mean Background:Oxides of Nitrogen (ug/m3),London Mean Background:Ozone (ug/m3),London Mean Background:PM10 Particulate (ug/m3),London Mean Background:PM2.5 Particulate (ug/m3),London Mean Background:Sulphur Dioxide (ug/m3)
0,Jan-08,NaN,55.5,NaN,29.5,25.0,14.7,4.2,NaN,42.3,NaN,36.9,18.8,.,3.6
1,Feb-08,NaN,75.9,NaN,20.3,39.5,28.8,7.6,NaN,60.2,NaN,26.4,31.9,.,6.7
2,Mar-08,NaN,55.6,NaN,40.1,21.6,12.3,3.9,NaN,39.8,NaN,50.2,15.5,.,2.3
3,Apr-08,NaN,61.8,NaN,37.9,28.7,20.5,4.5,NaN,44.0,NaN,50.1,21.7,.,3.2
4,May-08,NaN,62.9,NaN,46.3,34.6,27.5,4.6,NaN,44.1,NaN,60.5,29.5,16.6,4.3


In [206]:
data_air.drop(["London Mean Roadside:Nitric Oxide (ug/m3)","London Mean Background:Nitric Oxide (ug/m3)",
               "London Mean Roadside:Oxides of Nitrogen (ug/m3)", "London Mean Background:Oxides of Nitrogen (ug/m3)"
               ],axis=1,inplace=True)
data_air.rename(columns={"London Mean Roadside:Nitrogen Dioxide (ug/m3)":"NO2",
                         "London Mean Roadside:PM10 Particulate (ug/m3)":"PM10",
                        "London Mean Roadside:PM2.5 Particulate (ug/m3)":"PM25",
                        "London Mean Roadside:Sulphur Dioxide (ug/m3)":"SO2",
                        "London Mean Roadside:Ozone (ug/m3)" : "O3",
                        "London Mean Background:Nitrogen Dioxide (ug/m3)":"bg NO2",
                         "London Mean Background:PM10 Particulate (ug/m3)":"bg PM10",
                        "London Mean Background:PM2.5 Particulate (ug/m3)":"bg PM25",
                        "London Mean Background:Sulphur Dioxide (ug/m3)":"bg SO2",
                        "London Mean Background:Ozone (ug/m3)" : "bg O3",
                         },inplace=True)
data_air
data_air.set_index("Month")


,NO2,O3,PM10,PM25,SO2,bg NO2,bg O3,bg PM10,bg PM25,bg SO2
Month,,,,,,,,,,
Jan-08,55.5,29.5,25.0,14.7,4.2,42.3,36.9,18.8,.,3.6
Feb-08,75.9,20.3,39.5,28.8,7.6,60.2,26.4,31.9,.,6.7
Mar-08,55.6,40.1,21.6,12.3,3.9,39.8,50.2,15.5,.,2.3
Apr-08,61.8,37.9,28.7,20.5,4.5,44.0,50.1,21.7,.,3.2
May-08,62.9,46.3,34.6,27.5,4.6,44.1,60.5,29.5,16.6,4.3
...,...,...,...,...,...,...,...,...,...,...
Mar-19,42.6,34.9,20.9,11.3,4.9,25.7,52.7,16.6,10.9,2.3
Apr-19,46.2,38.1,32.3,22.8,12.4,31.5,54.7,29.2,23.1,2.8
May-19,39.7,35.7,19.5,10.7,11.5,22.4,53.7,15.3,10.9,1.5


In pollution data-set, the only relevant variables will be pollutant,the number of emissions, date, the city and the sector the emited the pollutant

In [207]:
data_pol.head()

,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
289510,United Kingdom,5.0,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,GB.EEA/13495.FACILITY,"Viridor Waste Management Ltd, Beddington Farml...",NaN,-0.142952,51.389667,NaN,Sutton,AIR,Chlorofluorocarbons (CFCs),11.10,2007,NaN
289511,United Kingdom,5.0,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,GB.EEA/13495.FACILITY,"Viridor Waste Management Ltd, Beddington Farml...",NaN,-0.142952,51.389667,NaN,Sutton,AIR,Hydrochlorofluorocarbons (HCFCs),4.88,2007,NaN
289512,United Kingdom,5.0,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,GB.EEA/13495.FACILITY,"Viridor Waste Management Ltd, Beddington Farml...",NaN,-0.142952,51.389667,NaN,Sutton,AIR,Methane (CH4),117000.00,2007,NaN
289806,United Kingdom,3.0,Mineral industry,3(e),"Installations for the manufacture of glass, in...",GB.EEA/31898.FACILITY,O-I Manufacturing UK,NaN,0.110430,51.783016,NaN,Harlow,AIR,Arsenic and compounds (as As),35.00,2007,NaN
289807,United Kingdom,3.0,Mineral industry,3(e),"Installations for the manufacture of glass, in...",GB.EEA/31898.FACILITY,O-I Manufacturing UK,NaN,0.110430,51.783016,NaN,Harlow,AIR,Nitrogen oxides (NOX),446000.00,2007,NaN


In [209]:
data_pol = data_pol[["eprtrSectorName","City","pollutant","emissions","reportingYear"]]
data_pol.rename(columns={"eprtrSectorName": "SectorName","reportingYear":"Year"},inplace=True)
data_pol.head()

,SectorName,City,pollutant,emissions,Year
289510,Waste and wastewater management,Sutton,Chlorofluorocarbons (CFCs),11.10,2007
289511,Waste and wastewater management,Sutton,Hydrochlorofluorocarbons (HCFCs),4.88,2007
289512,Waste and wastewater management,Sutton,Methane (CH4),117000.00,2007
289806,Mineral industry,Harlow,Arsenic and compounds (as As),35.00,2007
289807,Mineral industry,Harlow,Nitrogen oxides (NOX),446000.00,2007


For this analysis we won't consider the flow in the different roads, thus the only variable related to that,  will be "all_motor_vehicles" which includes the sum of all road flows

In [210]:
data_traf.head()

,count_point_id,year,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,6750,2014,6,London,96,Islington,A201,Major,Swinton St,Pentonville Rd,...,787,1728,204,74,132,15,6,18,449,11916
1,8352,2014,6,London,104,Lewisham,A2,Major,A20,A20,...,430,1415,258,36,58,6,11,22,390,9426
2,6807,2014,6,London,178,Kingston upon Thames,A240,Major,A3210,A307,...,175,1589,251,35,13,2,4,19,325,12951
3,27765,2014,6,London,57,Barnet,A5109,Major,Barnet Lane,A1000/A109,...,302,1651,246,12,12,0,1,1,273,19276
4,8488,2014,6,London,104,Lewisham,A20,Major,A20 Amersham Rd,A2 Parkfield Rd,...,656,1617,286,83,27,5,8,7,415,12821


In [213]:
data_traf = data_traf[["year","local_authority_name","all_motor_vehicles"]]
data_traf.head()

,year,local_authority_name,all_motor_vehicles
0,2014,Islington,11916
1,2014,Lewisham,9426
2,2014,Kingston upon Thames,12951
3,2014,Barnet,19276
4,2014,Lewisham,12821


From the health index data-set will only be withdrawn, related to health, the value and the Indicator name(area which the index is being applied), sice the numerator and denominator are only used in order to obtain the variable "value".

In [214]:
data_health.head()

,Area code,Area name,Value,Numerator,Denominator,Indicator name,Year
29,E06000032,Luton,"61,75566862",NaN,NaN,Physical activity,2015
36,E06000039,Slough,"59,60549999",NaN,NaN,Physical activity,2015
90,E07000066,Basildon,"65,4596393",NaN,NaN,Physical activity,2015
92,E07000068,Brentwood,"68,62717127",NaN,NaN,Physical activity,2015
94,E07000070,Chelmsford,"67,26048083",NaN,NaN,Physical activity,2015


In [217]:
data_health = data_health[["Area name","Value","Indicator name","Year"]]
data_health.head()

,Area name,Value,Indicator name,Year
29,Luton,"61,75566862",Physical activity,2015
36,Slough,"59,60549999",Physical activity,2015
90,Basildon,"65,4596393",Physical activity,2015
92,Brentwood,"68,62717127",Physical activity,2015
94,Chelmsford,"67,26048083",Physical activity,2015


In [ ]:
#code that is not used anymore
"""
def filterLocations(center, uk_data, radius_km):
    filtered_locations = pd.DataFrame()
    for a,k in uk_data.iterrows():
        loc = (k["Latitude"],k["Longitude"])
        distance = geodesic(center, loc).kilometers
        if distance <= radius_km:
            filtered_locations = pd.concat([filtered_locations, k.to_frame().T], ignore_index=True)
    return filtered_locations
"""


In [ ]:
#code that is not used anymore
"""
#coordinates of the center of london
lat_lon_london = (51.509865, -0.118092)
#cities of UK
uk_data = data_pol[data_pol["countryName"]== "United Kingdom"]
#take only the cities within a radius of 50 km
data_pol= filterLocations(lat_lon_london,uk_data,50)
data_pol
"""
